In [45]:
import os
import random
import sys
 
import tensorflow as tf
import cv2
from PIL import Image

In [46]:
def _int64_feature(value):
    """Wrapper for insert int64 feature into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
 
def _float_feature(value):
    """Wrapper for insert float features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
 
def _bytes_feature(value):
    """Wrapper for insert bytes features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [47]:
def _process_image_withoutcoder(filename):
    """
    利用cv2将filename指向的图片tostring
    """
 
    print("filename",filename)
    image = cv2.imread(filename)
 
    # transform data into string format
    image_data = image.tostring()
    assert len(image.shape) == 3
    height = image.shape[0]
    width = image.shape[1]
    assert image.shape[2] == 3
    # return string data and initial height and width of the image
    return image_data, height, width
 

def _convert_to_example_simple(image_example, image_buffer):
    """
        covert to tfrecord file
    Parameter
    ------------
        image_example: dict, an image example
        image_buffer: string, JPEG encoding of RGB image
    Return
    -----------
        Example proto
    """
    class_label = image_example['label']
 
    bbox = image_example['bbox']
    roi = [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']]
    landmark = [bbox['xlefteye'],bbox['ylefteye'],bbox['xrighteye'],bbox['yrighteye'],bbox['xnose'],bbox['ynose'],
                bbox['xleftmouth'],bbox['yleftmouth'],bbox['xrightmouth'],bbox['yrightmouth']]
 
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/encoded': _bytes_feature(image_buffer),
        'image/label': _int64_feature(class_label),
        'image/roi': _float_feature(roi),
        'image/landmark': _float_feature(landmark)
    }))
    return example
 
# 从图片和注释文件里加载数据并将其添加到TFRecord里
# 参数（变量）：filename:存有数据的字典；tfrecord_writer:用来写入TFRecord的writer
 
def _add_to_tfrecord(filename, image_example, tfrecord_writer):
    # print('---', filename)
 
    # imaga_data:转化为字符串的图片
    # height:图片原始高度
    # width:图片原始宽度
    # image_example：包含图片信息的字典
    # print(filename)
    image_data, height, width = _process_image_withoutcoder(filename)
    example = _convert_to_example_simple(image_example, image_data)
    tfrecord_writer.write(example.SerializeToString())  # 将imaga_data转化到image_example中并写入tfrecord
 
 
def _get_output_filename(output_dir,net):
    # 定义一下输出的文件名
 
    # return '%s/%s_%s_%s.tfrecord' % (output_dir, name, net, st)
    # st = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    # time.strftime() 函数接收以时间元组，并返回以可读字符串表示的当地时间，格式由参数format决定:time.strftime(format[, t]),用来输出当前时间
    # 返回的是'../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
    return 'data/%s/train_%s_landmark.tfrecord' % (output_dir,net)


In [48]:
def run(dataset_dir,net,output_dir,shuffling=False):
    """
    运行转换操作
    Args:
      dataset_dir: 数据集所在的数据集目录
      output_dir: 输出目录
    """
 
    # tfrecord name
    tf_filename = _get_output_filename(output_dir,net)  # '../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
 
    if tf.io.gfile.exists(tf_filename):  # tf.io.gfile模块提供了文件操作的API,包括文件的读取、写入、删除、复制等等
        print('Dataset files already exist. Exiting without re-creating them.')  # 判断是否存在同名文件
        return
 
    # 获得数据集，并打乱顺序
    dataset = new_get_dataset(dataset_dir)
    #print(dataset)
    # filenames = dataset['filename']
    if shuffling:
        tf_filename = tf_filename + '_shuffle'
        # random.seed(12345454)
        random.shuffle(dataset)  # 打乱dataset数据集的顺序
 
    # Process dataset files.
    # write the data to tfrecord
    print('lala')
    with tf.io.TFRecordWriter(tf_filename) as tfrecord_writer:
        for i, image_example in enumerate(dataset):  # 读取dataset的索引和内容
            if (i + 1) % 1 == 0:
                sys.stdout.write('\r>> %d/%d images has been converted' % (
                i + 1, len(dataset)))  # 输出“x00/ len(dataset) images has been converted”
            sys.stdout.flush()  # 以一定间隔时间刷新输出
            filename = image_example['filename']  # 赋值
            #print(filename)
            _add_to_tfrecord(filename, image_example, tfrecord_writer)
    # 最后，编写标签文件
    # labels_to_class_names = dict(zip(range(len(_CLASS_NAMES)), _CLASS_NAMES))
    # dataset_utils.write_label_file(labels_to_class_names, dataset_dir)
    print('\nFinished converting the MTCNN dataset!')
 


In [49]:
def get_dataset(dir):
    # 获取文件名字，标签和注释
    item = 'label-train%s.txt'%(dir)
 
    dataset_dir = os.path.join('data',dir, item)  # dataset_dir = '../../DATA/imglists/PNet/train_PNet_landmark.txt'
    # print(dataset_dir)
    imagelist = open(dataset_dir, 'r')  # 以只读的形式打开train_PNet_landmark.txt，并传入imagelist里面
 
    dataset = []  # 新建列表
    for line in imagelist.readlines():  # 按行读取imagelist里面的内容
        info = line.strip().split(' ')  # .strip().split()去除每一行首尾空格并且以空格为分隔符读取内容到info里面
        data_example = dict()  # 新建字典
        bbox = dict()
        data_example['filename'] = 'data/' + info[0]  # filename=info[0]
        # print(data_example['filename'])
        data_example['label'] = int(info[1])  # label=info[1]，info[1]的值有四种可能，1，0，-1，-2；分别对应着正、负、无关、关键点样本
        bbox['xmin'] = 0  # 初始化bounding box的值
        bbox['ymin'] = 0
        bbox['xmax'] = 0
        bbox['ymax'] = 0
        bbox['xlefteye'] = 0  # 初始化人脸坐标的值
        # bbox['ylefteye'] = 0
        # bbox['xrighteye'] = 0
        # bbox['yrighteye'] = 0
        # bbox['xnose'] = 0
        # bbox['ynose'] = 0
        # bbox['xleftmouth'] = 0
        # bbox['yleftmouth'] = 0
        # bbox['xrightmouth'] = 0
        # bbox['yrightmouth'] = 0
        if len(info) == 6:  # 当info的长度等于6时，表示此时的info是正样本或者无关样本
            bbox['xmin'] = float(info[2])
            bbox['ymin'] = float(info[3])
            bbox['xmax'] = float(info[4])
            bbox['ymax'] = float(info[5])
        # if len(info) == 12:  # 当info的长度等于12时，表示此时的info是landmark样本
        #     bbox['xlefteye'] = float(info[2])
        #     bbox['ylefteye'] = float(info[3])
        #     bbox['xrighteye'] = float(info[4])
        #     bbox['yrighteye'] = float(info[5])
        #     bbox['xnose'] = float(info[6])
        #     bbox['ynose'] = float(info[7])
        #     bbox['xleftmouth'] = float(info[8])
        #     bbox['yleftmouth'] = float(info[9])
        #     bbox['xrightmouth'] = float(info[10])
        #     bbox['yrightmouth'] = float(info[11])
 
        data_example['bbox'] = bbox  # 将bounding box值传入字典
        dataset.append(data_example)  # 将data_example字典内容传入列表dataset
 
    return dataset  # 返回的是dataset，datase是个列表，但里面每个元素都是一个字典，每个字典都含有3个key，分别是filename、label和bounding box

In [50]:
def read_data(line):
    info = line.strip().split(' ')  # .strip().split()去除每一行首尾空格并且以空格为分隔符读取内容到info里面
    data_example = dict()  # 新建字典
    bbox = dict()
    data_example['filename'] = 'data/' + info[0]  # filename=info[0]
    # print(data_example['filename'])
    data_example['label'] = int(info[1])  # label=info[1]，info[1]的值有四种可能，1，0，-1，-2；分别对应着正、负、无关、关键点样本
    bbox['xmin'] = 0  # 初始化bounding box的值
    bbox['ymin'] = 0
    bbox['xmax'] = 0
    bbox['ymax'] = 0
    bbox['xlefteye'] = 0  # 初始化人脸坐标的值
    bbox['ylefteye'] = 0
    bbox['xrighteye'] = 0
    bbox['yrighteye'] = 0
    bbox['xnose'] = 0
    bbox['ynose'] = 0
    bbox['xleftmouth'] = 0
    bbox['yleftmouth'] = 0
    bbox['xrightmouth'] = 0
    bbox['yrightmouth'] = 0
    if len(info) == 6:  # 当info的长度等于6时，表示此时的info是正样本或者无关样本
        bbox['xmin'] = float(info[2])
        bbox['ymin'] = float(info[3])
        bbox['xmax'] = float(info[4])
        bbox['ymax'] = float(info[5])
    if len(info) == 12:  # 当info的长度等于12时，表示此时的info是landmark样本
        bbox['xlefteye'] = float(info[2])
        bbox['ylefteye'] = float(info[3])
        bbox['xrighteye'] = float(info[4])
        bbox['yrighteye'] = float(info[5])
        bbox['xnose'] = float(info[6])
        bbox['ynose'] = float(info[7])
        bbox['xleftmouth'] = float(info[8])
        bbox['yleftmouth'] = float(info[9])
        bbox['xrightmouth'] = float(info[10])
        bbox['yrightmouth'] = float(info[11])
 
    data_example['bbox'] = bbox  # 将bounding box值传入字典
    return data_example

In [51]:
def new_get_dataset(dir):
    item_pos = 'pos_%s.txt'%(dir)
    item_neg = 'neg_%s.txt'%(dir)
    item_part = 'part_%s.txt'%(dir)
 
    dataset_dir_pos = os.path.join('data',dir, item_pos)  # dataset_dir = '../../DATA/imglists/PNet/train_PNet_landmark.txt'
    dataset_dir_neg = os.path.join('data',dir, item_neg)
    dataset_dir_part = os.path.join('data',dir, item_part)
    dataset_dir_landmark = os.path.join('data/48/landmark_48_aug.txt')
    
    check_pos = True
    check_neg = True
    check_part = True
    check_landmark = True
    
    pos_list = open(dataset_dir_pos, 'r')
    neg_list = open(dataset_dir_neg, 'r')
    part_list = open(dataset_dir_part, 'r')
    landmark_list = open(dataset_dir_landmark, 'r')
    
    dataset = []
    
    while check_pos or check_neg or check_part or check_landmark:
        pos_line = pos_list.readline()
        neg_line = neg_list.readline()
        part_line = part_list.readline()
        landmark_line = landmark_list.readline()
        
        if not pos_line:
            pos_list.close()
            check_pos = False
            pos_list = open(dataset_dir_pos, 'r')
            pos_line = pos_list.readline()
        if not neg_line:
            neg_list.close()
            check_neg = False
            neg_list = open(dataset_dir_neg, 'r')
            neg_line = neg_list.readline()
        if not part_line:
            part_list.close()
            check_part = False
            part_list = open(dataset_dir_part, 'r')
            part_line = part_list.readline()
        if not landmark_line:
            landmark_list.close()
            check_landmark = False
            landmark_list = open(dataset_dir_landmark, 'r')
            landmark_line = landmark_list.readline()
        
        dataset.append(read_data(pos_line))
        dataset.append(read_data(neg_line))
        dataset.append(read_data(part_line))
        dataset.append(read_data(landmark_line))
    pos_list.close()
    neg_list.close()
    part_list.close()
    landmark_list.close()
    
    return dataset

In [52]:

if __name__ == '__main__':
    dir = '48'
    net = 'ONet'
    output_directory = '48'
    run(dir,net,output_directory,shuffling=False)

lala
>> 1/24004 images has been convertedfilename data/48/positive/0.jpg
>> 2/24004 images has been convertedfilename data/48/negative/0.jpg
>> 3/24004 images has been convertedfilename data/48/part/0.jpg
>> 4/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/0.jpg
>> 5/24004 images has been convertedfilename data/48/positive/1.jpg
>> 6/24004 images has been convertedfilename data/48/negative/1.jpg
>> 7/24004 images has been convertedfilename data/48/part/1.jpg
>> 8/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/1.jpg
>> 9/24004 images has been convertedfilename data/48/positive/2.jpg
>> 10/24004 images has been convertedfilename data/48/negative/2.jpg
>> 11/24004 images has been convertedfilename data/48/part/2.jpg
>> 12/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/2.jpg
>> 13/24004 images has been convertedfilename data/48/positive/3.jpg
>> 14/24004 images has been convertedfilename data/48/negative/3.jpg
>> 15

>> 114/24004 images has been convertedfilename data/48/negative/28.jpg
>> 115/24004 images has been convertedfilename data/48/part/28.jpg
>> 116/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/28.jpg
>> 117/24004 images has been convertedfilename data/48/positive/29.jpg
>> 118/24004 images has been convertedfilename data/48/negative/29.jpg
>> 119/24004 images has been convertedfilename data/48/part/29.jpg
>> 120/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/29.jpg
>> 121/24004 images has been convertedfilename data/48/positive/30.jpg
>> 122/24004 images has been convertedfilename data/48/negative/30.jpg
>> 123/24004 images has been convertedfilename data/48/part/30.jpg
>> 124/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/30.jpg
>> 125/24004 images has been convertedfilename data/48/positive/31.jpg
>> 126/24004 images has been convertedfilename data/48/negative/31.jpg
>> 127/24004 images has been convertedfilena

/opt/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.


filename data/48/negative/33.jpg
>> 135/24004 images has been convertedfilename data/48/part/33.jpg
>> 136/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/33.jpg
>> 137/24004 images has been convertedfilename data/48/positive/34.jpg
>> 138/24004 images has been convertedfilename data/48/negative/34.jpg
>> 139/24004 images has been convertedfilename data/48/part/34.jpg
>> 140/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/34.jpg
>> 141/24004 images has been convertedfilename data/48/positive/35.jpg
>> 142/24004 images has been convertedfilename data/48/negative/35.jpg
>> 143/24004 images has been convertedfilename data/48/part/35.jpg
>> 144/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/35.jpg
>> 145/24004 images has been convertedfilename data/48/positive/36.jpg
>> 146/24004 images has been convertedfilename data/48/negative/36.jpg
>> 147/24004 images has been convertedfilename data/48/part/36.jpg
>> 148/24004 im

>> 246/24004 images has been convertedfilename data/48/negative/61.jpg
>> 247/24004 images has been convertedfilename data/48/part/61.jpg
>> 248/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/61.jpg
>> 249/24004 images has been convertedfilename data/48/positive/62.jpg
>> 250/24004 images has been convertedfilename data/48/negative/62.jpg
>> 251/24004 images has been convertedfilename data/48/part/62.jpg
>> 252/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/62.jpg
>> 253/24004 images has been convertedfilename data/48/positive/63.jpg
>> 254/24004 images has been convertedfilename data/48/negative/63.jpg
>> 255/24004 images has been convertedfilename data/48/part/63.jpg
>> 256/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/63.jpg
>> 257/24004 images has been convertedfilename data/48/positive/64.jpg
>> 258/24004 images has been convertedfilename data/48/negative/64.jpg
>> 259/24004 images has been convertedfilena

...

>> 23083/24004 images has been convertedfilename data/48/part/736.jpg
>> 23084/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/765.jpg
>> 23085/24004 images has been convertedfilename data/48/positive/743.jpg
>> 23086/24004 images has been convertedfilename data/48/negative/5771.jpg
>> 23087/24004 images has been convertedfilename data/48/part/737.jpg
>> 23088/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/766.jpg
>> 23089/24004 images has been convertedfilename data/48/positive/744.jpg
>> 23090/24004 images has been convertedfilename data/48/negative/5772.jpg
>> 23091/24004 images has been convertedfilename data/48/part/738.jpg
>> 23092/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/767.jpg
>> 23093/24004 images has been convertedfilename data/48/positive/745.jpg
>> 23094/24004 images has been convertedfilename data/48/negative/5773.jpg
>> 23095/24004 images has been convertedfilename data/48/part/739.jpg
>> 230

>> 23189/24004 images has been convertedfilename data/48/positive/769.jpg
>> 23190/24004 images has been convertedfilename data/48/negative/5797.jpg
>> 23191/24004 images has been convertedfilename data/48/part/763.jpg
>> 23192/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/792.jpg
>> 23193/24004 images has been convertedfilename data/48/positive/770.jpg
>> 23194/24004 images has been convertedfilename data/48/negative/5798.jpg
>> 23195/24004 images has been convertedfilename data/48/part/764.jpg
>> 23196/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/793.jpg
>> 23197/24004 images has been convertedfilename data/48/positive/771.jpg
>> 23198/24004 images has been convertedfilename data/48/negative/5799.jpg
>> 23199/24004 images has been convertedfilename data/48/part/765.jpg
>> 23200/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/794.jpg
>> 23201/24004 images has been convertedfilename data/48/positive/772.jpg
>>

>> 23295/24004 images has been convertedfilename data/48/part/789.jpg
>> 23296/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/818.jpg
>> 23297/24004 images has been convertedfilename data/48/positive/796.jpg
>> 23298/24004 images has been convertedfilename data/48/negative/5824.jpg
>> 23299/24004 images has been convertedfilename data/48/part/790.jpg
>> 23300/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/819.jpg
>> 23301/24004 images has been convertedfilename data/48/positive/797.jpg
>> 23302/24004 images has been convertedfilename data/48/negative/5825.jpg
>> 23303/24004 images has been convertedfilename data/48/part/791.jpg
>> 23304/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/820.jpg
>> 23305/24004 images has been convertedfilename data/48/positive/798.jpg
>> 23306/24004 images has been convertedfilename data/48/negative/5826.jpg
>> 23307/24004 images has been convertedfilename data/48/part/792.jpg
>> 233

>> 23401/24004 images has been convertedfilename data/48/positive/822.jpg
>> 23402/24004 images has been convertedfilename data/48/negative/5850.jpg
>> 23403/24004 images has been convertedfilename data/48/part/816.jpg
>> 23404/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/845.jpg
>> 23405/24004 images has been convertedfilename data/48/positive/823.jpg
>> 23406/24004 images has been convertedfilename data/48/negative/5851.jpg
>> 23407/24004 images has been convertedfilename data/48/part/817.jpg
>> 23408/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/846.jpg
>> 23409/24004 images has been convertedfilename data/48/positive/824.jpg
>> 23410/24004 images has been convertedfilename data/48/negative/5852.jpg
>> 23411/24004 images has been convertedfilename data/48/part/818.jpg
>> 23412/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/847.jpg
>> 23413/24004 images has been convertedfilename data/48/positive/825.jpg
>>

>> 23508/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/871.jpg
>> 23509/24004 images has been convertedfilename data/48/positive/11.jpg
>> 23510/24004 images has been convertedfilename data/48/negative/5877.jpg
>> 23511/24004 images has been convertedfilename data/48/part/4.jpg
>> 23512/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/872.jpg
>> 23513/24004 images has been convertedfilename data/48/positive/12.jpg
>> 23514/24004 images has been convertedfilename data/48/negative/5878.jpg
>> 23515/24004 images has been convertedfilename data/48/part/5.jpg
>> 23516/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/873.jpg
>> 23517/24004 images has been convertedfilename data/48/positive/13.jpg
>> 23518/24004 images has been convertedfilename data/48/negative/5879.jpg
>> 23519/24004 images has been convertedfilename data/48/part/6.jpg
>> 23520/24004 images has been convertedfilename data/48/train_ONet_landmark_aug/874.